In [22]:
import pandas as pd
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
import numpy as np
from typing import Tuple
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import DataLoader, TensorDataset
import torch
from transformers import BertTokenizer, BertModel

In [23]:
TRC_FILE:str = "./trc.csv"
TDVACH_FILE:str = "./tdvach.csv"
RPT_FILE:str = "./rpt.csv"

In [24]:
def get_pd(filePath: str)->pd.DataFrame | None:
    if os.path.exists(filePath) and os.path.isfile(filePath):
        return pd.read_csv(filePath, index_col=0)
    match filePath:
        case "./trc.csv":
            df = pd.read_json("hf://datasets/AlexSham/Toxic_Russian_Comments/train.jsonl", lines=True)
        case "./tdvach.csv":
            df = pd.read_csv("hf://datasets/marriamaslova/toxic_dvach/labeled.csv")
        case "./rpt.csv":
            df = pd.read_csv("hf://datasets/s-nlp/ru_paradetox_toxicity/train.tsv", sep="\t")
        case _:
            return None
    df.to_csv(filePath)
    return df

In [25]:
df1 = get_pd(TRC_FILE)
print(df1.head())
print("*"*10)
print(df1['label'].value_counts())
print("*"*10)
print(df1[df1['label'] == 1].head(5))

                                                text  label
0  видимо в разных регионах называют по разному ,...      0
1  понятно что это нарушение правил, писать капсл...      1
2                какие классные, жизненные стихи....      0
3                 а и правда-когда его запретили?...      0
4  в соленой воде вирусы живут .ученые изучали со...      0
**********
label
0    183316
1     40145
Name: count, dtype: int64
**********
                                                 text  label
1   понятно что это нарушение правил, писать капсл...      1
5   правильно! это же тихановская 26 лет растила и...      1
13                                   на хуй, безликая      1
16                дебилов хватает.надо было с головой      1
30                  умник хуев. у каждого своё мнение      1


In [26]:
df2 = get_pd(TDVACH_FILE)
print(df2.head())
print("*"*10)
print(df2['toxic'].value_counts())
print("*"*10)
print(df2[df2['toxic'] == 1].head(5))

                                             comment  toxic
0               Верблюдов-то за что? Дебилы, бл...\n    1.0
1  Хохлы, это отдушина затюканого россиянина, мол...    1.0
2                          Собаке - собачья смерть\n    1.0
3  Страницу обнови, дебил. Это тоже не оскорблени...    1.0
4  тебя не убедил 6-страничный пдф в том, что Скр...    1.0
**********
toxic
0.0    9586
1.0    4826
Name: count, dtype: int64
**********
                                             comment  toxic
0               Верблюдов-то за что? Дебилы, бл...\n    1.0
1  Хохлы, это отдушина затюканого россиянина, мол...    1.0
2                          Собаке - собачья смерть\n    1.0
3  Страницу обнови, дебил. Это тоже не оскорблени...    1.0
4  тебя не убедил 6-страничный пдф в том, что Скр...    1.0


In [27]:
df3 = get_pd(RPT_FILE)
print(df3.head())
print("*"*10)
print(df3['toxic'].value_counts())
print("*"*10)
print(df3[df3['toxic'] == 1].head(5))

                                             neutral  toxic
0  Вот и довели себя до сахарного диабета второго...  False
1  Мойщик машину пеной залил, а хозяин виноват? П...  False
2                Какой совет может дать эта девушка?  False
3                    Зачем брать топор и убивать то?  False
4  Такой же как и Дегтярев. Другими словами оба р...  False
**********
toxic
False    4848
True     1506
Name: count, dtype: int64
**********
                                              neutral  toxic
11                     Дима! Их еще называли шестерки   True
15  Из-за тебя глупца потом стоять, потому что ты ...   True
19  Людей за бабки на улицы вывели чтобы гопника о...   True
26        глупый пытается попасть в элиту этого мира?   True
30     У тебя колорад сбился узел, помолчи пожалуйста   True


In [28]:
df2['toxic'] = df2['toxic'].astype(int)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  14412 non-null  object
 1   toxic    14412 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 337.8+ KB


In [29]:
df3['toxic'] = df3['toxic'].astype(int)
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6354 entries, 0 to 6353
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   neutral  6354 non-null   object
 1   toxic    6354 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 148.9+ KB


In [30]:
def combine_dataframes(
    df1:pd.DataFrame, 
    df2:pd.DataFrame,
    df3:pd.DataFrame,
)-> Tuple[list[object], list[object], list[int], list[int]]:
    x_1 = df1['text'].values
    y_1 = df1['label'].values
    x_1_train, x_1_test, y_1_train, y_1_test = train_test_split(x_1, y_1, test_size=0.2)
    
    x_2 = df2['comment'].values
    y_2 = df2['toxic'].values
    x_2_train, x_2_test, y_2_train, y_2_test = train_test_split(x_2, y_2, test_size=0.2)
    
    x_3 = df3['neutral'].values
    y_3 = df3['toxic'].values
    x_3_train, x_3_test, y_3_train, y_3_test = train_test_split(x_3, y_3, test_size=0.2)
    
    x_train = np.concatenate((x_1_train, x_2_train, x_3_train))
    x_test = np.concatenate((x_1_test, x_2_test, x_3_test))
    y_train = np.concatenate((y_1_train, y_2_train, y_3_train))
    y_test = np.concatenate((y_1_test, y_2_test, y_3_test))
    return x_train, x_test, y_train, y_test
    

In [31]:
X_train, X_test, y_train, y_test = combine_dataframes(df1, df2, df3)

In [32]:
all_words = " ".join(X_train).split()

# Используем set для нахождения уникальных слов
unique_words = set(all_words)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")
def text_to_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

X_embeddings = np.array([text_to_embedding(text) for text in tqdm(X_train, desc="Processing texts")])

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Processing texts:   1%|          | 1063/195380 [01:07<2:31:00, 21.45it/s]

KeyboardInterrupt: 

Processing texts:   1%|          | 1063/195380 [01:19<2:31:00, 21.45it/s]

In [ ]:
X_tensor = torch.tensor(X_embeddings, dtype=torch.float32)